In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np
import pickle
import os
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
import sklearn
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
import pickle

In [2]:
def load_data():
    column_labels = ["time", "x", "y", "z", "activity"]
    data = pd.DataFrame()
    temp=[]
    for i in range(1,16):
        df = pd.read_csv(str(i) + ".csv", names = column_labels)
        temp.append(df)
    data = pd.concat(temp)
    return data

In [3]:
data = load_data()

In [4]:
data.head(5)

,time,x,y,z,activity
0,0.0,1502,2215,2153,1
1,1.0,1667,2072,2047,1
2,2.0,1611,1957,1906,1
3,3.0,1601,1939,1831,1
4,4.0,1643,1965,1879,1


In [5]:
data = data[data.activity != 0]

In [6]:
def normalize(col):
    mean = np.mean(col,axis = 0)
    sig = np.std(col,axis = 0)
    return (col - mean)/sig

data['x'] = normalize(data['x'])
data['y'] = normalize(data['y'])
data['z'] = normalize(data['z'])

In [9]:
data.head()

,time,x,y,z,activity
0,0.0,-4.360297,-1.669479,1.931520,1
1,1.0,-2.878360,-3.096472,0.809722,1
2,2.0,-3.381321,-4.244054,-0.682480,1
3,3.0,-3.471135,-4.423675,-1.476205,1
4,4.0,-3.093915,-4.164222,-0.968221,1


In [10]:
def windows(data, size):
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += (size / 2)
        
def segment_signal(data,window_size = 200):
    segments = np.empty((0,window_size,3))
    labels = np.empty((0))
    for (start, end) in windows(data["time"], window_size):
        start = int(start)
        end = int(end)
        x = data["x"][start:end]
        y = data["y"][start:end]
        z = data["z"][start:end]
        if(len(data["time"][start:end]) == window_size):
            segments = np.vstack([segments,np.dstack([x,y,z])])
            labels = np.append(labels,stats.mode(data["activity"][start:end])[0][0])
    return segments, labels



In [11]:
# segments, labels = segment_signal(data)

In [12]:
# labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
# reshaped = segments.reshape(len(segments), 1,200, 3)

In [13]:
# pickle.dump( segments, open( "segments.p", "wb" ) )
# pickle.dump( labels, open( "labels.p", "wb" ) )
# pickle.dump( reshaped, open( "reshaped_segments.p", "wb" ) )

In [ ]:
# segments = pickle.load( open( "segments.p", "rb" ) )
# labels = pickle.load( open( "labels.p", "rb" ) )
# reshaped= pickle.load( open( "reshaped_segments.p", "rb" ) )

In [96]:
segments.shape

(19230, 200, 3)

In [98]:
# split = np.random.rand(len(segments)) < 0.70
# train_x = reshaped[split]
# train_y = labels[split]
# test_x = reshaped[~split]
# test_y = labels[~split]

In [95]:
# print (train_x.shape , train_y.shape)
# print (test_x.shape , test_y.shape)

In [99]:
split = np.random.rand(len(segments)) < 0.70
X_train = segments[split]
y_train = labels[split]
X_test = segments[~split]
y_test = labels[~split]

In [73]:
batch_size = 600       # Batch size
seq_len = 200          # Number of steps
learning_rate = 0.0005
epochs = 200

n_classes = 7
n_channels = 3

In [102]:
X_train.shape

(13451, 200, 3)

In [75]:
inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

In [76]:
cconv1 = tf.layers.conv1d(inputs=inputs_, filters=18, kernel_size=2, strides=1, 
                         padding='same', activation = tf.nn.relu)
pool_1 = tf.layers.max_pooling1d(inputs=cconv1, pool_size=4, strides=4, padding='same')

cconv2 = tf.layers.conv1d(inputs=pool_1, filters=36, kernel_size=2, strides=1, 
                         padding='same', activation = tf.nn.relu)
pool_2 = tf.layers.max_pooling1d(inputs=cconv2, pool_size=4, strides=4, padding='same')

cconv3 = tf.layers.conv1d(inputs=pool_2, filters=72, kernel_size=2, strides=1, 
                         padding='same', activation = tf.nn.relu)
pool_3 = tf.layers.max_pooling1d(inputs=cconv3, pool_size=4, strides=4, padding='same')

In [90]:

flat = tf.reshape(pool_3, (-1, 2*72))
print(flat.get_shape())
flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
print(flat.get_shape())
logits = tf.layers.dense(flat, n_classes)
print(logits.get_shape())
# Cost function and optimizer

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
#     print(cost.get_shape())
optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
tf.summary.scalar("cost",cost)

# Accuracy

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
tf.summary.scalar("accuracy",accuracy)


(?, 144)
(?, 144)
(?, 7)


In [92]:
def get_batches(X, y, batch_size = 100):
    """ Return a generator for batches """
    n_batches = len(X) // batch_size
    X, y = X[:n_batches*batch_size], y[:n_batches*batch_size]

    # Loop over batches and yield
    for b in range(0, len(X), batch_size):
        yield X[b:b+batch_size], y[b:b+batch_size]

        
# def one_hot(labels, n_class = 7):
#     """ One-hot encoding """
#     expansion = np.eye(n_class)
#     y = expansion[:, labels-1].T
#     assert y.shape[1] == n_class, "Wrong number of labels!"
#     return y

In [84]:
test_acc = []
test_loss = []

train_acc = []
train_loss = []

# with graph.as_default():
saver = tf.train.Saver()

# with tf.Session(graph=graph) as sess:
# with tf.Session() as sess:

sess = tf.Session()
sess.run(tf.global_variables_initializer())
# writer = tf.summary.FileWriter("logs/", sess.graph)

iteration = 1

for e in range(epochs):
#     tf.set_random_seed(123)
    # Loop over batches
    for x,y in get_batches(X_train, y_train, batch_size):

        # Feed dictionary
        feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, learning_rate_ : learning_rate}

        # Loss
        loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
        train_acc.append(acc)
        train_loss.append(loss)

        # Print at each 5 iters
        if (iteration % 5 == 0):
            print("Epoch: {}/{}".format(e, epochs),
                  "Iteration: {:d}".format(iteration),
                  "Train loss: {:6f}".format(loss),
                  "Train acc: {:.6f}".format(acc))

        # Compute validation loss at every 10 iterations
        if (iteration%10 == 0):                
            val_acc_ = []
            val_loss_ = []

            for x_t, y_t in get_batches(X_test, y_test, batch_size):
                # Feed
                feed = {inputs_ : x_t, labels_ : y_t, keep_prob_ : 1.0}  

                # Loss
                loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
                val_acc_.append(acc_v)
                val_loss_.append(loss_v)

            # Print info
            print("Epoch: {}/{}".format(e, epochs),
                  "Iteration: {:d}".format(iteration),
                  "Testing loss NOW: {:6f}".format(np.mean(val_loss_)),
                  "Testing acc NOW: {:.6f}".format(np.mean(val_acc_)))

            # Store
            test_acc.append(np.mean(val_acc_))
            test_loss.append(np.mean(val_loss_))

        # Iterate 
        iteration += 1

    print("Optimization Finished!")
print("Ended!")

saver.save(sess,"cnn-saved/3_layer_har.ckpt")

InvalidArgumentError: logits and labels must be same size: logits_size=[1200,7] labels_size=[600,7]
	 [[Node: cost_9/SoftmaxCrossEntropyWithLogits = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](cost_9/Reshape, cost_9/Reshape_1)]]

Caused by op 'cost_9/SoftmaxCrossEntropyWithLogits', defined at:
  File "C:\Users\Wayne\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Wayne\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Wayne\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Wayne\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Wayne\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.inst